In [ ]:
import pandas as pd
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from imblearn.under_sampling import RandomUnderSampler
from sentence_transformers import SentenceTransformer

## 1. Negative classifier

In [ ]:
full_df = pd.read_csv('data_for_filtering/job_with_compliance_keywords.csv')
negative_df = full_df[full_df['label'] == 0].reset_index()

In [ ]:
bert_model = SentenceTransformer('all-MiniLM-L6-v2')

n_repeat = 10
models = []
for i in tqdm(range(n_repeat), total=n_repeat):
        model = XGBClassifier()
        model.load_model(f"xgboost_models/model{i}.json")
        models.append(model)

In [ ]:
negative_df['y_predict'] = 0
new_X = bert_model.encode(negative_df['clean_job_description'])

print('start predicting negative_df...')
for i in range(n_repeat):
    negative_df['y_predict']  += models[i].predict(new_X)
negative_df['y_predict'] /= n_repeat
negative_df['y_predict_label'] = negative_df['y_predict'] >= 0.5
refined_negative_df = negative_df[negative_df['y_predict_label'] == 0]
print('refined negatives:', refined_negative_df.shape, 'total negatives:', negative_df.shape)
# negative_df.to_csv('negative_identifier_output_6232022.csv', index=False)
refined_negative_df.to_csv('negative_identifier_output_TEST.csv', index=False)